### Preprocessing
This module performs data transformation in preparation for the customer churn model training.

1. It reads raw data in CSV from GCS
2. Performs some basic transformations and
3. Persists to BigQuery

Copyright 2022 Google LLC

 Licensed under the Apache License, Version 2.0 (the "License");
 you may not use this file except in compliance with the License.
 You may obtain a copy of the License at

      http://www.apache.org/licenses/LICENSE-2.0

 Unless required by applicable law or agreed to in writing, software
 distributed under the License is distributed on an "AS IS" BASIS,
 WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
 See the License for the specific language governing permissions and
 limitations under the License.

In [ ]:
spark

In [ ]:
import sys
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from datetime import datetime
import random

In [ ]:
# 1a. Arguments
pipelineID = random.randint(1, 10000)
projectNbr = "YOUR_PROJECT_NBR"
projectID = "YOUR_PROJECT_ID"
displayPrintStatements = True

In [ ]:
# 1b. Variables 
bqDatasetNm = f"{projectID}.customer_churn_ds"
appBaseName = "customer-churn-model"
appNameSuffix = "preprocessing"
appName = f"{appBaseName}-{appNameSuffix}"
scratchBucketUri = f"s8s-spark-bucket-{projectNbr}/{appBaseName}/pipelineId-{pipelineID}/{appNameSuffix}"
sourceBucketUri = f"gs://s8s_data_bucket-{projectNbr}/customer_churn_train_data.csv"
bigQueryTargetTableFQN = f"{bqDatasetNm}.training_data"
pipelineExecutionDt = datetime.now().strftime("%Y%m%d%H%M%S")

In [ ]:
# 1c. Display input and output
if displayPrintStatements:
    print("Starting preprocessing for the *Customer Churn* experiment")
    print(".....................................................")
    print(f"The datetime now is - {pipelineExecutionDt}")
    print(" ")
    print("INPUT PARAMETERS-")
    print(f"....pipelineID={pipelineID}")
    print(f"....projectID={projectID}")
    print(f"....projectNbr={projectNbr}")
    print(f"....displayPrintStatements={displayPrintStatements}")
    print(" ")
    print("EXPECTED SETUP-")  
    print(f"....BQ Dataset={bqDatasetNm}")
    print(f"....Source Data={sourceBucketUri}")
    print(f"....Scratch Bucket for BQ connector=gs://s8s-spark-bucket-{projectNbr}") 
    print("OUTPUT-")
    print(f"....BigQuery Table={bigQueryTargetTableFQN}")
    print(f"....Sample query-")
    print(f"....SELECT * FROM {bigQueryTargetTableFQN} WHERE pipeline_id='{pipelineID}' LIMIT 10" )
  

In [ ]:
# 2. Spark Session creation
print('....Initializing spark & spark configs')
spark = SparkSession.builder.appName(appName).getOrCreate()

# Spark configuration setting for writes to BigQuery
spark.conf.set("parentProject", projectID)
spark.conf.set("temporaryGcsBucket", scratchBucketUri)

# Add Python modules
sc.addPyFile(f"gs://s8s_code_bucket-{projectNbr}/pyspark/common_utils.py")
import common_utils

In [ ]:
# 3. Read raw data in GCS into a Spark Dataframe
print('....Read source data')
rawChurnDF = spark.read.options(inferSchema = True, header= True).csv(sourceBucketUri)

In [ ]:
# 4. View the data
if displayPrintStatements:
    print(rawChurnDF.count())
    rawChurnDF.show(2)

In [ ]:
# 5. Profile the data
if displayPrintStatements:
    rawChurnDF.describe().show()

In [ ]:
# 6. Check for spaces, nulls in monthly & total charges
print('....Exploratory Data Analysis')
if displayPrintStatements:
    rawChurnDF.createOrReplaceTempView("base_customer_churn")
    spark.sql("select count(*) from base_customer_churn where MonthlyCharges is null or MonthlyCharges=' '").show(5)
    spark.sql("select count(*) from base_customer_churn where TotalCharges is null or TotalCharges=' '").show(5)


In [ ]:
# 7. Replace spaces, space with null values in the TotalCharges and MonthlyCharges columns
print('....Replace space, nulls with None')
spaceReplacedDF = common_utils.fnReplaceSpaceWithNone(rawChurnDF)
if displayPrintStatements:
    print(spaceReplacedDF.count())

In [ ]:
# 8. Replace non-numeric values values in the TotalCharges and MonthlyCharges columns
print('....Replace non-numeric values in numeric columns with null')
nanReplacedDF = common_utils.fnReplaceNotANumberWithNone(spaceReplacedDF)
if displayPrintStatements:
    print(nanReplacedDF.count())

In [ ]:
# 9. Drop rows with null in columns
print('....Drop nulls')
nullDroppedDF = nanReplacedDF.na.drop()
if displayPrintStatements:
    print(nullDroppedDF.count())

In [ ]:
# 10. Replace 'No internet service' across columns to 'No'
print('....Replace -No internet service across columns- to -No-')
partiallyProcessedDF = common_utils.fnReplaceWithNoForInternetService(nullDroppedDF)
if displayPrintStatements:
    print(partiallyProcessedDF.count())

In [ ]:
# 11. Add a bin/bucket category for tenure range using Spark SQL and write transformed to dataframe
print('....Add a bin for tenure')
modelTrainingReadyDF = common_utils.fnAddBinForTenure(partiallyProcessedDF, False, spark)
if displayPrintStatements:
    print(modelTrainingReadyDF.count())

In [ ]:
# 12. Run summary statistics
if displayPrintStatements:
    modelTrainingReadyDF.describe().show()

In [ ]:
# 13. Print schema
modelTrainingReadyDF.printSchema()

In [ ]:
# 14. Format column names for consistency (title case to DB style & lowercase)
print('....Format column names for consistency')
persistDF = modelTrainingReadyDF.select("customerID", "gender", "SeniorCitizen", "Partner", "Dependents", "tenure", "Tenure_Group", "PhoneService", "MultipleLines", "InternetService", "OnlineSecurity", "OnlineBackup", "DeviceProtection", "TechSupport", "StreamingTV", "StreamingMovies", "Contract", "PaperlessBilling", "PaymentMethod", "MonthlyCharges", "TotalCharges","Churn") \
                                .toDF("customer_id", "gender", "senior_citizen", "partner", "dependents", "tenure", "tenure_group", "phone_service", "multiple_lines", "internet_service", "online_security", "online_backup", "device_protection", "tech_support", "streaming_tv", "streaming_movies", "contract", "paperless_billing", "payment_method", "monthly_charges", "total_charges","churn") \
                                .withColumn("pipeline_id", lit(pipelineID).cast("string")) \
                                .withColumn("pipeline_execution_dt", lit(pipelineExecutionDt)) 

persistDF.printSchema()


In [ ]:
# 15. Persist training dataset to a table in BQ with the pipeline ID and execution date for traceability
print('....Persist to BQ')  
persistDF.write.format('bigquery') \
.mode("append")\
.option('table', bigQueryTargetTableFQN) \
.save()